In [1]:
import json
import yfinance as yf
import random
random.seed(0)
from pytickersymbols import PyTickerSymbols

def is_valid_ticker(ticker):
    """Check if a ticker exists by retrieving 1 day of historical data."""
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty  # Valid if data is not empty
    except:
        return False  # Invalid ticker

stock_data = PyTickerSymbols()
stocks = list(stock_data.get_dow_jones_nyc_yahoo_tickers())
stocks = [s for s in stocks if is_valid_ticker(s)]
print("Available stocks: ", len(stocks))

experiment_data = {}

for experiment_id in range(6):
    experiment_data[experiment_id] = []
    for n_stocks in range(2, 15):
        data_point = {}
        sampled_stocks = random.sample(stocks, n_stocks)
        start = "2015-01-01"
        end = "2025-01-01"

        data = yf.download(sampled_stocks, start=start, end=end, progress=False)
        prices_now = data["Close"].iloc[-1]
        data_point["stocks"] = sampled_stocks
        data_point["start"] = start
        data_point["end"] = end
        data_point["n_stocks"] = n_stocks
        
        experiment_data[experiment_id].append(data_point)

with open("experiments_data.json", "w") as f:
    json.dump(experiment_data, f, indent=4)

$GS-PK: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Available stocks:  30
YF.download() has changed argument auto_adjust default to True
